In [ ]:
#from metrics import recall_k, ndcg_k, repeat_score_item, repeat_score_user

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!nvidia-smi

Tue May 23 05:50:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install wandb

In [ ]:
import wandb
wandb.login(key='4da15e357372a40c1c71a28c87306f19bf380d80')

In [ ]:
def wandb_start(config, run_name):
    wandb.init(project="NBR_instacart", config=config)
    wandb.run.name = run_name

In [ ]:
config = {
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu')
}

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import os
import time
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
import json
import itertools
import sys
from scipy.sparse import csr_matrix, vstack
from scipy import sparse

In [ ]:
basket_count_min = 3
min_item_count = 5
history_len = 20

In [ ]:
# '''
# Reads the raw files, renames columns, last basket as test and the rest as train.
# No additional preprocessing steps.
# '''

# input_file_path = '/kaggle/input/data-ok/transaction_data.csv'  # сюда пути куда этот файл, я его я прям скину с таким же названием
# train_baskets_file_path = '/kaggle/working/train_baskets1.csv'  #  сюда тоже пути, это для новых файло, они сделаются
# test_baskets_file_path = '/kaggle/working/test_baskets1.csv'    #  сюда тоже пути, это для новых файло, они сделаются
# valid_baskets_file_path = '/kaggle/working/valid_baskets1.csv'  #  сюда тоже пути, это для новых файло, они сделаются

# df = pd.read_csv(input_file_path)
# print(df.shape)
# df['date'] = df['DAY'].astype(int)
# df['basket_id'] = df['BASKET_ID']
# df['item_id'] = df['PRODUCT_ID'].astype(str)
# df['user_id'] = df['household_key'].astype(str)

# processed_df = df[['date','basket_id','user_id','item_id']].drop_duplicates()
# print(processed_df.shape)
# print(processed_df.nunique())
# last_baskets = processed_df[['user_id','basket_id','date']].drop_duplicates() \
#     .groupby('user_id').apply(lambda grp: grp.nlargest(1, 'date'))
# last_baskets.index = last_baskets.index.droplevel()
# test_baskets = pd.merge(last_baskets, processed_df, how='left')


# train_baskets = pd.concat([processed_df,test_baskets]).drop_duplicates(keep=False)

# all_users = list(set(test_baskets['user_id'].tolist()))
# valid_indices = np.random.choice(range(len(all_users)),int(0.5*len(all_users)),
#                                  replace=False)
# valid_users = [all_users[i] for i in valid_indices]

# valid_baskets = test_baskets[test_baskets['user_id'].isin(valid_users)]
# test_baskets = test_baskets[~test_baskets['user_id'].isin(valid_users)]

# print(valid_baskets.shape)
# print(test_baskets.shape)
# print(train_baskets.shape)

# print(valid_baskets.nunique())
# print(test_baskets.nunique())
# print(train_baskets.nunique())

# train_baskets.to_csv(train_baskets_file_path,index=False)
# test_baskets.to_csv(test_baskets_file_path,index=False)
# valid_baskets.to_csv(valid_baskets_file_path,index=False) 

In [ ]:
# multihot_dict = {}
# for user_id, items_list in zip(train_baskets.groupby(['user_id'])['item_id'].apply(list).reset_index()['user_id'],
#                                train_baskets.groupby(['user_id'])['item_id'].apply(list).reset_index()['item_id']):
#     l1 = sorted(list(set([dataset.item_id_mapper[int(item)] for item in items_list if int(item) in dataset.item_id_mapper])))
#     ll2 = np.zeros(dataset.num_items-1)
#     ll2[[(i - 1) for i in l1]] = 1
#     multihot_dict[dataset.user_id_mapper[int(user_id)]] = ll2
    #[1 if i in l1 else 0 for i in np.array(list(dataset.id_item_mapper.keys()))]
    #print(len(list(multihot_dict.keys())))

In [ ]:
# multihot_dict = {}
# for user_id, items_list in zip(train_baskets.groupby(['user_id'])['item_id'].apply(list).reset_index()['user_id'],
#                                train_baskets.groupby(['user_id'])['item_id'].apply(list).reset_index()['item_id']):
#     l1 = sorted(list(set([dataset.item_id_mapper[item] for item in items_list if item in dataset.item_id_mapper])))
#     multihot_dict[dataset.user_id_mapper[user_id]] = [1 if i in l1 else 0 for i in np.array(list(dataset.id_item_mapper.keys()))]
#     print(len(list(multihot_dict.keys())))

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# preparind dataset for torch.dataset class
class PreDataset():
    def __init__(self, path_train,path_val, path_test, dataset, history_len, basket_count_min=0,\
                                                         min_item_count = 0):
        self.basket_count_min = basket_count_min
        self.min_item_count = min_item_count
        self.history_len = history_len
        
        self.train_baskets = pd.read_csv(path_train)
        self.valid_baskets = pd.read_csv(path_val)
        self.test_baskets = pd.read_csv(path_test)
        
        basket_per_user = self.train_baskets[['user_id','basket_id']].drop_duplicates() \
            .groupby('user_id').agg({'basket_id':'count'}).reset_index()
        self.test_users = basket_per_user[basket_per_user['basket_id'] >= self.basket_count_min]['user_id'].tolist()
    
        print("number of test users:", len(self.test_users))
        
        self.model_name = 'data/dunnhumby_cj/'+dataset+ '_recanet'
        self.dataset = dataset
        self.all_items = self.train_baskets[['item_id']].drop_duplicates()['item_id'].tolist()
        self.all_users = self.train_baskets[['user_id']].drop_duplicates()['user_id'].tolist()
        self.num_items = len(self.all_items) +1
        self.num_users = len(self.all_users) +1

        print("items:", self.num_items)
        item_counts = self.train_baskets.groupby(['item_id']).size().to_frame(name = 'item_count').reset_index()
        item_counts = item_counts[item_counts['item_count']>= min_item_count]
        item_counts_dict = dict(zip(item_counts['item_id'],item_counts['item_count']))
        print("filtered items:", len(item_counts_dict))
        self.num_items = len(item_counts_dict) +1
        self.item_id_mapper = {} 
        self.id_item_mapper = {}
        self.user_id_mapper = {}
        self.id_user_mapper = {}

        counter = 0
        for i in range(len(self.all_items)):
            if self.all_items[i] in item_counts_dict:
                self.item_id_mapper[self.all_items[i]] = counter+1
                self.id_item_mapper[counter+1] = self.all_items[i]
                counter+=1
        for i in range(len(self.all_users)):
            self.user_id_mapper[self.all_users[i]] = i+1
            self.id_user_mapper[i+1] = self.all_users[i]
            
            
    def create_train_data(self):
        #if os.path.isfile('/content/drive/MyDrive/dunn2/'+ str(20) + '_train_users.npy'):
        if os.path.isfile('/content/drive/MyDrive/dunn2/'+ str(20) + '_train_users.npy'):
            print('Data allready in use')
            train_users = np.load('/content/drive/MyDrive/dunn2/'+ str(self.history_len) + '_train_users.npy')
            train_items = np.load('/content/drive/MyDrive/dunn2/'+ str(self.history_len) + '_train_items.npy')
      
            #train_history2 = np.load('Downloads/dunnhumby3/dunnhumby_upd/'+ str(self.history_len) + '_train_history2.npy')
            train_history2 = sparse.load_npz('/content/drive/MyDrive/dunn2/'+ str(self.history_len) + '_train_history2.npz')
            train_labels = np.load('/content/drive/MyDrive/dunn2/'+ str(self.history_len)+ '_train_labels.npy')
        #if os.path.isfile('/content/drive/MyDrive/dunnhumby_cj/dunnhumby_cj_recanet_20_test_history2.npy'):
        #if os.path.isfile('/kaggle/input/npy-files/dunnhumby_cj_recanet_20_test_history2.npy'):
        #if os.path.isfile(self.model_name +'_' + str(self.history_len) + '_train_users.npy'):
        #    print('Data allready in use')
        #    train_users = np.load('/content/drive/MyDrive/dunnhumby_cj/dunnhumby_cj_recanet_20' + '_train_users.npy')
        #    train_items = np.load('/content/drive/MyDrive/dunnhumby_cj/dunnhumby_cj_recanet_20' + '_train_items.npy')
        #    #train_history = np.load(self.model_name +'_' + str(self.history_len) + '_train_history.npy')
        #    train_history2 = np.load('/content/drive/MyDrive/dunnhumby_cj/dunnhumby_cj_recanet_20' + '_train_history2.npy')
        #    train_labels = np.load('/content/drive/MyDrive/dunnhumby_cj/dunnhumby_cj_recanet_20'+ '_train_labels.npy')
            return train_items,train_users, train_history2, train_labels
        #if os.path.isfile(self.model_name +'_' + str(self.history_len) + '_train_users.npy'):
        #    print('Data allready in use')
        #    train_users = np.load(self.model_name +'_' + str(self.history_len) + '_train_users.npy')
        #    train_items = np.load(self.model_name +'_' + str(self.history_len) + '_train_items.npy')
      
        #    train_history2 = np.load(self.model_name +'_' + str(self.history_len) + '_train_history2.npy')
        #    train_labels = np.load(self.model_name +'_' + str(self.history_len)+ '_train_labels.npy')
        #    return train_items,train_users, train_history2 , train_labels
        row_counts = 0
        basket_items = self.train_baskets.groupby(['basket_id'])['item_id'].apply(list).reset_index()
        basket_items_dict = dict(zip(basket_items['basket_id'],basket_items['item_id']))
        basket_items_dict['null'] = []

        user_baskets = self.train_baskets[['user_id','date','basket_id']].drop_duplicates().\
            sort_values(['user_id','date'],ascending=True).groupby(['user_id'])['basket_id'].apply(list).reset_index()

        user_baskets_dict = dict(zip(user_baskets['user_id'],user_baskets['basket_id']))


        train_users = []
        train_items = []
        train_history = []
        train_history2 = []
        train_labels = []
        print('num users:', len(self.test_users))
        
        for c, user in tqdm(enumerate(self.test_users)):
            if c % 1000 ==1:
                print(c , 'user passed')

            baskets = user_baskets_dict[user]
            item_seq = {}
            for i, basket in enumerate(baskets):
                for item in basket_items_dict[basket]:
                    if item not in self.item_id_mapper:
                        continue
                    if item not in item_seq:
                        item_seq[item] = []
                    item_seq[item].append(i)


            for i in range(max(0,len(baskets)-50), len(baskets)):
                label_basket = baskets[i]
                all_history_baskets = baskets[:i]
                items = []
                for basket in all_history_baskets:
                    for item in basket_items_dict[basket]:
                        items.append(item)
                items = list(set(items))
                for item in items:
                    if item not in self.item_id_mapper:
                        continue
                    index = np.argmax(np.array(item_seq[item])>=i)
                    if np.max(np.array(item_seq[item])) < i:
                        index = len(item_seq[item])
                    input_history = item_seq[item][:index].copy()
                    if len(input_history) ==0:
                        continue
                    if len(input_history) ==1 and input_history[0]==-1:
                        continue
                    while len(input_history) < self.history_len:
                        input_history.insert(0,-1)
                    #real_input_history = []
                    #for x in input_history:
                    #    if x == -1:
                    #        real_input_history.append(0)
                    #    else:
                    #        real_input_history.append(i-x)
                    real_input_history2 = []
                    for j,x in enumerate(input_history[:-1]):
                        if x == -1:
                            real_input_history2.append(0)
                        else:
                            real_input_history2.append(input_history[j+1]-input_history[j])
                    real_input_history2.append(i-input_history[-1])
                    train_users.append(self.user_id_mapper[user])
                    train_items.append(self.item_id_mapper[item])
                    
#                     real = csr_matrix(real_input_history2[-self.history_len:])
#                     if type(train_history2) == list and len(train_history2) < 24721:
#                         train_history2.append(real)
#                     elif type(train_history2) == list and len(train_history2) == 24721:
#                         train_history2 = vstack(train_history2)
#                         train_history2 = vstack((train_history2, real))
#                     else:
#                         train_history2 = vstack((train_history2, real))
                        
                    #train_history.append(real_input_history[-self.history_len:])
                    train_history2.append(real_input_history2[-self.history_len:])
                    #print(item, basket_items_dict[label_basket])
                    train_labels.append(float(item in basket_items_dict[label_basket]))

                    row_counts +=1
            print(row_counts)

        train_items = np.array(train_items)
        train_users = np.array(train_users)
        #train_history = np.array(train_history)
        train_history2 = np.array(train_history2)
        train_labels = np.array(train_labels)
        random_indices = np.random.choice(range(len(train_items)), len(train_items),replace=False)
        train_items = train_items[random_indices]
        train_users = train_users[random_indices]
        #train_history = train_history[random_indices]
        train_history2 = train_history2[random_indices]
        train_labels = train_labels[random_indices]  
        
        train_history2 = csr_matrix(train_history2)

        #np.save(self.model_name +'_' + str(self.history_len) + '_train_items.npy',train_items)
        np.save('Downloads/dunn/'+ str(self.history_len) + '_train_items.npy', train_items)
        np.save('Downloads/dunn/'+ str(self.history_len) + '_train_users.npy', train_users)
      
       # np.save('Downloads/dunn/'+ str(self.history_len) + '_train_history2.npy', train_history2)
        sparse.save_npz('Downloads/dunn/'+ str(self.history_len) + '_train_history2.npz', train_history2)
        np.save('Downloads/dunn/'+ str(self.history_len) + '_train_labels.npy', train_labels)

        return train_items,train_users, train_history2 , train_labels
    
    
    def create_test_data(self,test_data='test'):
        
        if os.path.isfile('/content/drive/MyDrive/dunn2/'+ str(self.history_len)+ '_'+test_data+'_users.npy'):
            test_users = np.load('/content/drive/MyDrive/dunn2/'+ str(self.history_len) + '_'+test_data+'_users.npy')
            test_items = np.load('/content/drive/MyDrive/dunn2/'+ str(self.history_len) + '_'+test_data+'_items.npy')
        
            #test_history2 = sparse.load_npz('Downloads/dunnhumby3/'+ str(self.history_len) + '_'+test_data+'_history2.npz')
            test_history2 = np.load('/content/drive/MyDrive/dunn2/'+ str(self.history_len) + '_'+test_data+'_history2.npy')
            test_labels = np.load('/content/drive/MyDrive/dunn2/'+ str(self.history_len) + '_'+test_data+'_labels.npy')
        #if os.path.isfile('/content/drive/MyDrive/dunnhumby_cj/dunnhumby_cj_recanet_20_test_history2.npy'):
        #if os.path.isfile('/kaggle/input/npy-files/dunnhumby_cj_recanet_20_test_history2.npy'):
        #if os.path.isfile(self.model_name +'_' + str(self.history_len)+ '_'+test_data+'_users.npy'):
        #    test_users = np.load('/content/drive/MyDrive/dunnhumby_cj/dunnhumby_cj_recanet_20' + '_'+test_data+'_users.npy')
        #    test_items = np.load('/content/drive/MyDrive/dunnhumby_cj/dunnhumby_cj_recanet_20' + '_'+test_data+'_items.npy')
            #test_history = np.load(self.model_name +'_' + str(self.history_len) + '_'+test_data+'_history.npy')
        #    test_history2 = np.load('/content/drive/MyDrive/dunnhumby_cj/dunnhumby_cj_recanet_20' + '_'+test_data+'_history2.npy')
        #    test_labels = np.load('/content/drive/MyDrive/dunnhumby_cj/dunnhumby_cj_recanet_20' + '_'+test_data+'_labels.npy')
            return test_items,test_users,test_history2, test_labels
        #if os.path.isfile(self.model_name +'_' + str(self.history_len)+ '_'+test_data+'_users.npy'):
        #    test_users = np.load(self.model_name +'_' + str(self.history_len) + '_'+test_data+'_users.npy')
        #    test_items = np.load(self.model_name +'_' + str(self.history_len) + '_'+test_data+'_items.npy')
        
        #    test_history2 = np.load(self.model_name +'_' + str(self.history_len) + '_'+test_data+'_history2.npy')
        #    test_labels = np.load(self.model_name +'_' + str(self.history_len) + '_'+test_data+'_labels.npy')
        #    return test_items, test_users,  test_history2, test_labels

        train_basket_items = self.train_baskets.groupby(['basket_id'])['item_id'].apply(list).reset_index()
        train_basket_items_dict = dict(zip(train_basket_items['basket_id'],train_basket_items['item_id']))

        train_user_baskets = self.train_baskets[['user_id','date','basket_id']].drop_duplicates(). \
            sort_values(['user_id','date'],ascending=True).groupby(['user_id'])['basket_id'].apply(list).reset_index()
        train_user_baskets_dict = dict(zip(train_user_baskets['user_id'],train_user_baskets['basket_id']))

        train_user_items = self.train_baskets[['user_id','item_id']].drop_duplicates().groupby(['user_id'])['item_id'] \
            .apply(list).reset_index()
        train_user_items_dict = dict(zip(train_user_items['user_id'],train_user_items['item_id']))

        test_user_items = None
        if test_data == 'test':
            test_user_items = self.test_baskets.groupby(['user_id'])['item_id'].apply(list).reset_index()
        else:
            test_user_items = self.valid_baskets.groupby(['user_id'])['item_id'].apply(list).reset_index()
        test_user_items_dict = dict(zip(test_user_items['user_id'],test_user_items['item_id']))

        test_users = []
        test_items = []
        test_history = []
        test_history2 = []
        test_labels = []

        train_basket_items_dict['null'] = []
        for c,user in tqdm(enumerate(test_user_items_dict)):
            if user not in train_user_baskets_dict:
                continue
            if c % 100 ==1:
                print(c , 'user passed')
                #break

            baskets = train_user_baskets_dict[user]
            item_seq = {}
            for i, basket in enumerate(baskets):
                for item in train_basket_items_dict[basket]:
                    if item not in self.item_id_mapper:
                        continue
                    if item not in item_seq:
                        item_seq[item] = []
                    item_seq[item].append(i)


            label_items = test_user_items_dict[user]

            items = list(set(train_user_items_dict[user]))

            #print(len(history_baskets))
            for item in items:#train_user_items_dict[user]:
                if item not in self.item_id_mapper:
                    continue
                input_history = item_seq[item][-self.history_len:]
                if len(input_history) ==0:
                    continue
                if len(input_history) ==1 and input_history[0]==-1:
                    continue
                while len(input_history) < self.history_len:
                    input_history.insert(0,-1)
                #real_input_history = []
                #for x in input_history:
                #    if x == -1:
                #        real_input_history.append(0)
                #    else:
                #        real_input_history.append(len(baskets)-x)

                real_input_history2 = []
                for j,x in enumerate(input_history[:-1]):
                    if x == -1:
                        real_input_history2.append(0)
                    else:
                        real_input_history2.append(input_history[j+1]-input_history[j])
                real_input_history2.append(len(baskets)-input_history[-1])
                test_users.append(self.user_id_mapper[user])
                test_items.append(self.item_id_mapper[item])
                    
                #test_history.append(real_input_history)
                test_history2.append(real_input_history2)
                test_labels.append(float(item in label_items))

        test_items = np.array(test_items)
        test_users = np.array(test_users)
        #test_history = np.array(test_history)
        test_history2 = np.array(test_history2)
        test_labels = np.array(test_labels)

        np.save('Downloads/dunn/'+ str(self.history_len) + '_'+test_data+'_items.npy',test_items)
        np.save('Downloads/dunn/'+ str(self.history_len) + '_'+test_data+'_users.npy',test_users)
       
        np.save('Downloads/dunn/'+ str(self.history_len) + '_'+test_data+'_history2.npy',test_history2)
        #sparse.save_npz('Downloads/dunnhumby/'+ str(self.history_len) + '_'+test_data+'_history2.npz', test_history2)
        np.save('Downloads/dunn/'+ str(self.history_len) + '_'+test_data+'_labels.npy',test_labels)

        return test_items, test_users, test_history2, test_labels

In [ ]:
# path_train = f'/kaggle/input/dunnhumby-dataset/train_baskets.csv'
# path_test = f'/kaggle/input/dunnhumby-dataset/test_baskets.csv'
# path_val = f'/kaggle/input/dunnhumby-dataset/valid_baskets.csv'

In [ ]:
class RCNDataset(Dataset):
    def __init__(self, model, mode):

        self.mode = mode
        if self.mode=='train':
            self.x1, self.x2, self.x4, self.y = model.create_train_data()
        elif self.mode=='valid':
            self.x1, self.x2, self.x4, self.y = model.create_test_data(test_data='valid')
        elif self.mode=='test':
            self.x1, self.x2, self.x4, self.y = model.create_test_data(test_data='test')
        else: 
            print('Mode error')

        #self.x1, self.x2, self.x3, self.x4, self.y = torch.tensor(self.x1).long(), torch.tensor(self.x2).long(),\
        #            torch.tensor(self.x3), torch.tensor(self.x4), torch.tensor(self.y)
        if self.mode == 'train':
            self.x1, self.x2, self.y = torch.tensor(self.x1).long(), torch.tensor(self.x2).long(),\
              torch.FloatTensor(self.y)
        else:
            self.x1, self.x2, self.x4, self.y = torch.tensor(self.x1).long(), torch.tensor(self.x2).long(),\
                     torch.FloatTensor(self.x4), torch.FloatTensor(self.y)

    def __len__(self):
        return len(self.x1)

    def __getitem__(self, idx):
        if self.mode == 'train':
            return self.x1[idx], self.x2[idx], torch.FloatTensor(self.x4[idx].toarray()), self.y[idx]
        else:
            return self.x1[idx], self.x2[idx], self.x4[idx], self.y[idx]

## loading to device
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
history_len=20
item_embed_size=128
user_embed_size=32

h1 = 128
h2 = 128
h3 = 128
h4 = 128
h5 = 128
dataset_name = 'du'

In [ ]:
#!python -m wget -nc https://www.dropbox.com/s/5aac9i8z5uiewrn/transaction_data.csv.zip?dl=0 -O transaction_data.zip
# if the link does not work, you can download the dataset manually from https://disk.yandex.ru/d/nuyCNaDrE1Bq0w
#!python -m unzip -n transaction_data.zip

In [ ]:
#!python create_dunnhumby_cj_data.py

In [ ]:
# path_train = '/kaggle/working/train_baskets1.csv'  # для файлов что выше сделались
# path_val = '/kaggle/working/test_baskets1.csv'
# path_test = '/kaggle/working/valid_baskets1.csv'

In [ ]:
path_train = '/content/drive/MyDrive/baskets/train_baskets-git.csv'  # для файлов что выше сделались
path_val = '/content/drive/MyDrive/baskets/valid_baskets-git.csv'
path_test = '/content/drive/MyDrive/baskets/test_baskets-git.csv'

In [ ]:
# path_train = '/content/drive/MyDrive/instacart_baskets/train_baskets_inst.csv'  # для файлов что выше сделались
# path_val = '/content/drive/MyDrive/instacart_baskets/valid_baskets_inst.csv'
# path_test = '/content/drive/MyDrive/instacart_baskets/test_baskets_inst.csv'

In [ ]:
dataset = PreDataset(path_train,path_val, path_test, dataset=dataset_name, history_len=history_len,basket_count_min=3, min_item_count = 5)

number of test users: 2483
items: 91764
filtered items: 36963


In [ ]:
train_baskets = pd.read_csv(path_train)
test_baskets = pd.read_csv(path_test)
valid_baskets = pd.read_csv(path_val)

In [ ]:
test_baskets.shape, valid_baskets.shape, train_baskets.shape

((15372, 4), (14099, 4), (2566261, 4))

In [ ]:
test_users = dataset.test_users

In [ ]:
len(test_users)

2483

In [ ]:
max_bucket_len = max(train_baskets.groupby('basket_id').item_id.count())
max_num_baskets = max(train_baskets.groupby('user_id').basket_id.nunique())

In [ ]:
def gp_topfreq(train, test_users, n=max_num_baskets):
    result = {}
    top_popular = train.item_id.value_counts().index.tolist()[:max_bucket_len]
    train = train.sort_values(by='date')
    
    for user in tqdm(test_users):
        items = train[train.user_id==user]
        dates = items.date.unique()[-n:]
        top_personal = items[items.date.isin(dates)].item_id.value_counts().index.tolist()[:max_bucket_len]
        
        if len(top_personal)<max_bucket_len:
            gp_top = (top_personal + top_popular)[:max_bucket_len]
            result[user] = gp_top
        else:
            result[user] = top_personal
        
    return result

In [ ]:
res = gp_topfreq(train_baskets, test_users, n=50)

  0%|          | 0/2483 [00:00<?, ?it/s]

In [ ]:
path_test

'/content/drive/MyDrive/baskets/test_baskets-git.csv'

In [ ]:
test_baskets = pd.read_csv(path_test)
user_test_baskets_df = test_baskets.groupby('user_id')['item_id'].apply(list).reset_index()
user_test_baskets_dict = dict(zip( user_test_baskets_df['user_id'],user_test_baskets_df['item_id']))

user_predictions = res
final_users = set(dataset.test_users).intersection(set(list(user_test_baskets_dict.keys())))
print('predictions ready',len(user_predictions))
print('number of final test users:',len(final_users))
for k in [5,10,20,'B']:
    print(k)
    recall_scores = {}
    ndcg_scores = {}
    #zero = 0
    for user in final_users:

        top_items = []
        if user in user_predictions:
            top_items = user_predictions[user]
        else:
            zero+=1

        if k == 'B':
            recall_scores[user] = recall_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
            ndcg_scores[user] = ndcg_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
        else:
            recall_scores[user] = recall_k(user_test_baskets_dict[user],top_items,k)
            ndcg_scores[user] = ndcg_k(user_test_baskets_dict[user],top_items,k)
    #print(zero)
    print('recall:',np.mean(list(recall_scores.values())))
    print('ndcg:',np.mean(list(ndcg_scores.values())))

predictions ready 2483
number of final test users: 1243
5
recall: 0.1122471587550507
ndcg: 0.1710109936246384
10
recall: 0.14732749868705472
ndcg: 0.14012473594393066
20
recall: 0.20017823183717073
ndcg: 0.11050241486255191
B
recall: 0.12149068839735666
ndcg: 0.13985002689423645


In [ ]:
res

In [ ]:
multihot_dict = {}
for user_id, items_list in zip(train_baskets.groupby(['user_id'])['item_id'].apply(list).reset_index()['user_id'],
                               train_baskets.groupby(['user_id'])['item_id'].apply(list).reset_index()['item_id']):
    l1 = sorted(list(set([dataset.item_id_mapper[int(item)] for item in items_list if int(item) in dataset.item_id_mapper])))
    ll2 = np.zeros(dataset.num_items-1)
    ll2[[(i - 1) for i in l1]] = 1
    multihot_dict[dataset.user_id_mapper[int(user_id)]] = ll2

In [ ]:
2048*5, 2048*4, 2048*3, 2048*2

(10240, 8192, 6144, 4096)

In [ ]:
x1.shape, x2.shape, x4.shape, y.shape

((643538,), (643538,), (643538, 20), (643538,))

In [ ]:
batch_size = 2048

train_dataset = RCNDataset(dataset, mode='train')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = RCNDataset(dataset, mode='valid')
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = RCNDataset(dataset, mode='test')
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Data allready in use


In [ ]:
len(train_loader.dataset), len(val_loader.dataset), len(test_loader.dataset)

(50176675, 643538, 658898)

In [ ]:
len(train_loader.dataset), len(val_loader.dataset), len(test_loader.dataset)

(178201010, 6466341, 6511587)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, kq_same=False, bias=True):
        super().__init__()
        """
        It has projection layer for getting keys, queries and values. Followed by attention.
        """
        self.d_model = d_model
        self.h = n_heads
        self.d_k = self.d_model // self.h
        self.kq_same = kq_same

        if not kq_same:
            self.q_linear = nn.Linear(d_model, d_model, bias=bias)
        self.k_linear = nn.Linear(d_model, d_model, bias=bias)
        self.v_linear = nn.Linear(d_model, d_model, bias=bias)

    def head_split(self, x):  # get dimensions bs * h * seq_len * d_k
        new_x_shape = x.size()[:-1] + (self.h, self.d_k)
        return x.view(*new_x_shape).transpose(-2, -3)

    def forward(self, q, k, v, mask=None):
        origin_shape = q.size()

        # perform linear operation and split into h heads
        if not self.kq_same:
            q = self.head_split(self.q_linear(q))
        else:
            q = self.head_split(self.k_linear(q))
        k = self.head_split(self.k_linear(k))
        v = self.head_split(self.v_linear(v))

        # calculate attention using function we will define next
        output = self.scaled_dot_product_attention(q, k, v, self.d_k, mask)

        # concatenate heads and put through final linear layer
        output = output.transpose(-2, -3).reshape(origin_shape)
        return output

    @staticmethod
    def scaled_dot_product_attention(q, k, v, d_k, mask=None):
        """
        This is called by Multi-head attention object to find the values.
        """
        scores = torch.matmul(q, k.transpose(-2, -1)) / d_k ** 0.5  # bs, head, q_len, k_len
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -np.inf)
        scores = (scores - scores.max()).softmax(dim=-1)
        scores = scores.masked_fill(torch.isnan(scores), 0)
        output = torch.matmul(scores, v)  # bs, head, q_len, d_k
        return output

In [ ]:
class TransformerLayer(nn.Module):
    def __init__(self, d_model, d_ff, n_heads, dropout=0, kq_same=False):
        super().__init__()
        """
        This is a Basic Block of Transformer. It contains one Multi-head attention object. 
        Followed by layer norm and position wise feedforward net and dropout layer.
        """
        # Multi-Head Attention Block
        self.masked_attn_head = MultiHeadAttention(d_model, n_heads, kq_same=kq_same)

        # Two layer norm layer and two dropout layer
        self.layer_norm1 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)

        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)

        self.layer_norm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, seq, mask=None):
        context = self.masked_attn_head(seq, seq, seq, mask)
        context = self.layer_norm1(self.dropout1(context) + seq)
        output = self.linear1(context).relu()
        output = self.linear2(output)
        output = self.layer_norm2(self.dropout2(output) + context)
        return output

In [ ]:
class Block(nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.dense = nn.Linear(dim_in, dim_out)
        self.dropout = nn.Dropout(0.2)
        #self.norm = nn.BatchNorm2d(dim_out)
        self.norm = nn.LayerNorm(dim_out, elementwise_affine = False)
        #self.dropout = nn.Dropout(0.2)
        self.act = nn.Tanh()
        
    def forward(self, x):
        return self.act(self.norm(self.dropout(self.dense(x))))

In [ ]:
encoder_layer = nn.TransformerEncoderLayer(d_model=6, nhead=6, batch_first=True)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=4)

In [ ]:
transformer_encoder(x14).mean(dim=1)

torch.Size([6144, 128])

In [ ]:
import torch
import torch.nn as nn

In [ ]:
torch.eye(4, 6).shape

torch.Size([4, 6])

In [ ]:
transformer_encoder(torch.eye(4, 6)).shape

torch.Size([4, 6])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class ReCaNet(nn.Module):
    def __init__(self, num_items, item_embed_size, num_users, user_embed_size, history_len, h1, h2, h3, h4, h5):
        super(ReCaNet, self).__init__()
        self.num_items = num_items
        self.item_embed_size = item_embed_size
        self.num_users = num_users
        self.user_embed_size = user_embed_size
        self.history_len = history_len
        self.h1 = h1
        self.h2 = h2
        self.h3 = h3
        self.h4 = h4
        self.h5 = h5

        self.item_embedding = nn.Embedding(self.num_items, self.item_embed_size)
        self.user_embedding = nn.Embedding(self.num_users, self.user_embed_size)
        
        #self.fc1 = nn.Linear(self.item_embed_size + 36963, self.h1)
            #self.item_embed_size + self.user_embed_size, self.h1)
        self.fc1 = nn.Linear(self.item_embed_size + + self.user_embed_size, self.h1)
        self.fc2 = nn.Linear(self.h1+1, self.h2)
        self.lstm1 = nn.LSTM(self.h2, self.h3, batch_first=True, num_layers=2)
        #self.lstm2 = nn.LSTM(self.h3, self.h4, batch_first=True)
        self.fc5 = nn.Linear(self.h4, self.h5)
        self.fc6 = nn.Linear(self.h4, self.h5)
        self.fc7 = nn.Linear(self.h5, 1)
        #self.transformer_block = nn.ModuleList([
        #    TransformerLayer(d_model=self.h1, d_ff=self.h1, n_heads=4,
        #                            dropout=0.1, kq_same=False)
        #    for _ in range(1)
        #])
        #self.layers = nn.Sequential(
        #        Block(36963, 2048), 
        #        Block(2048, 512), 
        #        Block(512, 128)
        #)
        # self.encoder_layer = nn.TransformerEncoderLayer(d_model=128, nhead=16, batch_first=True)
        # self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=8)

    def forward(self, input1, input2, input4):
        x1 = self.item_embedding(input1.to(torch.int64))
        x1 = x1.view(-1, self.item_embed_size) #1x128
       
        x2 = self.user_embedding(input2.to(torch.int64))
        #x2 = torch.tensor(np.array([multihot_dict[i.item()] for i in input2.to(torch.int64)])).long().to(torch.float32).to(device)
        #x2 = x2.view(-1, 36963)
                     #self.user_embed_size) #1x32
        x2 = x2.view(-1, self.user_embed_size)
        #x2 = self.layers(x2)
        # 1x5
        x4 = input4 # 1x5
        x4 = x4.unsqueeze(2)
        
        conc1 = torch.cat((x1, x2), axis=1)
        x11 = F.relu(self.fc1(conc1)) # 1*128

        x12 = x11.unsqueeze(1).repeat(1, self.history_len, 1)
        
        conc2 = torch.cat((x12, x4), 2)#.transpose(-2,-1)
        x14 = F.relu(self.fc2(conc2))
        #valid_his = (input4 > 0).long()
        
        ###    ! Начало вставки с трансформером
#         causality_mask = np.tril(np.ones((1, 1, 20, 20), dtype=np.int))
#         attn_mask = torch.from_numpy(causality_mask).to(device)

#         for block in self.transformer_block:
#             his_vectors = block(x14, attn_mask)

#         valid_his = (input4 > 0).long()
#         seq_lengths = torch.sum(valid_his, axis=1)
#         his_vectors = his_vectors * valid_his[:, :, None].float()

#         his_vector = his_vectors[torch.arange(len(seq_lengths)), seq_lengths - 1, :]
        
#         x22 = his_vector[:, None, :].squeeze(1)

        ###    ! Конец вставки с трансформером

        x21, (hx, cx) = self.lstm1(x14)             #Прежняя часть с LSTM
        x22 = hx[1]                                 #Прежняя часть с LSTM
       
        #x22 = self.transformer_encoder(x14).mean(dim=1)
        x = F.relu(self.fc5(x22))
        x = F.relu(self.fc6(x))
      
        output = self.fc7(x).view(-1) 
        output = torch.sigmoid(output)
        return output

In [ ]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# model_new = ReCaNet(num_items=dataset.num_items, item_embed_size=item_embed_size, num_users=dataset.num_users, 
#                 user_embed_size=user_embed_size, history_len = history_len, h1 = h1,h2 = h2, h3 = h3,
#                 h4 = h4, h5 = h5).to(device)

In [ ]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# model_new = ReCaNet(num_items=dataset.num_items, item_embed_size=item_embed_size, num_users=dataset.num_users, 
#                 user_embed_size=user_embed_size, history_len = history_len, h1 = h1,h2 = h2, h3 = h3,
#                 h4 = h4, h5 = h5).to(device)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_new = ReCaNet(num_items=dataset.num_items, item_embed_size=item_embed_size, num_users=dataset.num_users, 
                user_embed_size=user_embed_size, history_len = history_len, h1 = h1,h2 = h2, h3 = h3,
                h4 = h4, h5 = h5).to(device)

#path_best = '/kaggle/input/weights/epoch_5_recanet_model_dunnhumby.pth'
#model_new.load_state_dict(torch.load(path_best))

In [ ]:
#optimizer = torch.optim.Adam(model_new.parameters(), lr = 0.001)
#criterion = nn.BCELoss(reduction='mean')

In [ ]:
wandb_start(config, 'recanet_instacart')

In [ ]:
wandb.watch(model_new)

[]

In [ ]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

In [ ]:
def epoch_recall(model, y_true ,y_pred, predictions, user_valid_baskets_dict, valid_users, valid_items):
    recall_scores = []
    for user in user_valid_baskets_dict:
        top_items = []
        if user in dataset.user_id_mapper:
            user_id = dataset.user_id_mapper[user]
            indices = np.argwhere(valid_users == user_id)
            item_scores = np.array(y_pred)[indices].flatten()
            item_ids = valid_items[indices].flatten()

            item_score_dic = {}
            for i, item_id in enumerate(item_ids):
                item_score_dic[dataset.id_item_mapper[item_id]] = item_scores[i]
            sorted_item_scores = sorted(item_score_dic.items(), key= lambda x: x[1], reverse = True)
            top_items = [x[0] for x in sorted_item_scores]
        recall_scores.append(recall_k(user_valid_baskets_dict[user],top_items,
                                              len(user_valid_baskets_dict[user])))
            
    return np.mean(recall_scores)

In [ ]:
x1.shape, x2.shape, x4.shape

(torch.Size([6144]), torch.Size([6144]), torch.Size([6144, 20]))

In [ ]:
for x1, x2, x4, y in tqdm(train_loader):
    x1 = x1.to(device)
    x2 = x2.to(device)
    #x3 = x3.to(device)
    x4 = x4.squeeze(1)
    x4 = x4.to(device) 

    output, x14, x21, (hx, cx), valid_his = model_new(x1, x2, x4)
    break

  0%|          | 0/8167 [00:00<?, ?it/s]

In [ ]:
output.shape, x14.shape, x21.shape, (hx, cx)

(torch.Size([6144]), torch.Size([6144, 20, 128]))

In [ ]:
#output.shape, causality_mask.shape, attn_mask.shape, x14.shape

In [ ]:
def train(model_new, epochs, calc_recall=True, checkpoint=False):
    torch.cuda.empty_cache()
    #clear_output(wait=True)
    loss_train = []
    loss_val = []
    metric_val = []
    metric_train = []

    recall_score = []
   
    best_model = None
    best_recall = 0

    parameters = filter(lambda p: p.requires_grad, model_new.parameters())
    optimizer = torch.optim.Adam(parameters, lr=0.001)
    criterion = nn.BCELoss(reduction='mean')

    #validation
    user_valid_baskets_df = dataset.valid_baskets.groupby('user_id')['item_id'].apply(list).reset_index()
    user_valid_baskets_dict = dict(zip( user_valid_baskets_df['user_id'],user_valid_baskets_df['item_id']))
    valid_items, valid_users, valid_history2 ,valid_labels = dataset.create_test_data('valid')

    
    for i in tqdm(range(epochs)):
        print(f'Epoch {i}')
        loss_train_epoch = []
        loss_val_epoch = []
        metric_val_epoch = []
        metric_train_epoch = []

        model_new.train()
        for x1, x2, x4, y in tqdm(train_loader):
            x1 = x1.to(device)
            x2 = x2.to(device)
            #x3 = x3.to(device)
            x4 = x4.squeeze(1)
            x4 = x4.to(device) 
            optimizer.zero_grad()
            
            y_pred = model_new(x1, x2, x4)
            predictions = [round(value) for value in y_pred.flatten().tolist()]
        
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            wandb.log({'train loss':loss.item()
                        })
            
            loss_train_epoch.append(loss.item())
            metric_train_epoch.append(accuracy_score(y.detach().cpu().numpy(), predictions))
        
        loss_train.append(np.mean(loss_train_epoch))
        metric_train.append(np.mean(metric_train_epoch))
        print(f'loss_train_epoch_{i}', np.mean(loss_train_epoch))
        print(f'metric_train_epoch_{i}', np.mean(metric_train_epoch))

        model_new.eval()
        
        full_y = []
        full_predictions = []
        pred_scores = []
        for x1, x2,  x4, y in val_loader:
            with torch.no_grad():
                x1 = x1.to(device)
                x2 = x2.to(device)
                #x3 = x3.to(device)
                x4 = x4.to(device) 
                y_pred = model_new(x1, x2, x4)
                
                loss = criterion(y_pred, y)
                wandb.log({'test loss':loss.item()
                        })

                predictions = [round(value) for value in y_pred.flatten().tolist()]
                full_y.extend(y.detach().cpu())
                pred_scores.extend(y_pred.detach().cpu())
                full_predictions.extend(predictions)

                loss_val_epoch.append(loss.item())
                metric_val_epoch.append(accuracy_score(y.detach().cpu().numpy(), predictions))
              
        print(f'loss_val_epoch_{i}', np.mean(loss_val_epoch))
        print(f'metric_val_epoch_{i}', np.mean(metric_val_epoch))
        accuracy = accuracy_score(full_y, full_predictions)
        print("Accuracy epoch: %.2f%%" % (accuracy * 100.0))
        
        if calc_recall:
            recall_per_epoch = epoch_recall(model_new,full_y,pred_scores, full_predictions, user_valid_baskets_dict, \
                                             valid_users, valid_items)
            recall_score.append(recall_per_epoch)

            if recall_per_epoch > best_recall:
                best_model = model_new
                best_recall = recall_per_epoch
                path_best = f'/kaggle/working/epoch_{i}_recanet_best_model_dunnhumby.pth'
                # '/kaggle/working/train_baskets.csv'
                #f'/content/drive/MyDrive/epoch_{i}_recanet_best_model_dunnhumby.pth'
                torch.save(best_model.state_dict(), path_best) #saving model with best loss

        best_model = model_new       
        path_best = f'/content/drive/MyDrive/ckpt_instacart/epoch_{i}_recanet_model_inst.pth'
        torch.save(model_new.state_dict(), path_best) #saving model with best loss

        loss_val.append(np.mean(loss_val_epoch))
        metric_val.append(np.mean(metric_val_epoch))
        
        # отрисовка графиков
        #clear_output(wait=True)
        #plot_training(loss_train, loss_val,metric_train, metric_val, recall_score ,i=i, calc_recall=calc_recall)

        if checkpoint:
            #path = f'checkpoints/epoch{i}_recanet_dunnhumby_20.pth'
            #torch.save(model.state_dict(), path)
            np.save(f'/content/drive/MyDrive/ckpt_instacart/loss_train.npy', loss_train)
            np.save(f'/content/drive/MyDrive/ckpt_instacart/loss_val.npy', loss_val)
            np.save(f'/content/drive/MyDrive/ckpt_instacart/metric_train.npy', metric_train)
            np.save(f'/content/drive/MyDrive/ckpt_instacart/metric_val.npy', metric_val)
            #np.save('checkpoints/recall_score.npy', recall_score)
        
    if calc_recall:
        #print(recall_score)
        print('Best epoch:', np.argmax(np.array(recall_score))) 
    return best_model, loss_train, loss_val

In [ ]:
def predict(model, test_loader):
    
    test_items, test_users, test_history2, test_labels = dataset.create_test_data('test')
    preds = []
    pred_scores = []
    
    for x1, x2,  x4, y in tqdm(test_loader):
        with torch.no_grad():
            y_pred = model(x1, x2, x4)
            pred = [round(value) for value in y_pred.flatten().tolist()]
            preds.extend(pred)
            pred_scores.extend(y_pred.flatten().tolist())
            
   
    prediction_baskets = {}
    prediction_scores = {}
    for user in dataset.test_users:
        top_items = []
        if user in dataset.user_id_mapper:
            user_id = dataset.user_id_mapper[user]
            indices = np.argwhere(test_users == user_id)
            item_scores = np.array(pred_scores)[indices].flatten()
            item_ids = test_items[indices].flatten()
            item_score_dic = {}
            for i, item_id in enumerate(item_ids):
                item_score_dic[dataset.id_item_mapper[item_id]] = item_scores[i]
            sorted_item_scores = sorted(item_score_dic.items(), key= lambda x: x[1], reverse = True)
            top_items = [x[0] for x in sorted_item_scores]
            prediction_scores[user] = sorted_item_scores
            
        prediction_baskets[user] = top_items

    return prediction_baskets

In [ ]:
import numpy as np

def recall_k(y_true, y_pred, k ):
    a = len(set(y_pred[:k]).intersection(set(y_true)))
    b = len(set(y_true))
    return a/b


def ndcg_k(y_true, y_pred, k):
    a = 0
    for i,x in enumerate(y_pred[:k]):
        if x in y_true:
            a+= 1/np.log2(i+2)
    b = 0
    for i in range(k):#range(min(k,len(set(y_true)))):
        b +=1/np.log2(i+2)
    return a/b


def repeat_score_item(train_baskets):
    user_item = train_baskets.groupby(['user_id','item_id']).size().to_frame(name = 'item_count'). \
        reset_index()

    user_train_baskets_df = train_baskets[['user_id','basket_id']].drop_duplicates().groupby(['user_id'])\
        .size().reset_index()
    user_train_baskets_dict = dict(zip( user_train_baskets_df['user_id'],user_train_baskets_df[0]))


    user_baskets = train_baskets[['user_id','date','basket_id']].drop_duplicates(). \
        sort_values(['user_id','date'],ascending=True).groupby(['user_id'])['basket_id'].apply(list).reset_index()
    user_baskets_dict = dict(zip(user_baskets['user_id'],user_baskets['basket_id']))

    basket_items = train_baskets.groupby(['basket_id'])['item_id'].apply(list).reset_index()
    basket_items_dict = dict(zip(basket_items['basket_id'],basket_items['item_id']))

    rep_score = {}
    for user in user_baskets_dict:
        baskets = user_baskets_dict[user]
        rep_score[user] = 0
        for i in range(len(baskets)):
            next_basket = baskets[i]
            history_baskets = baskets[:i]

            history_items = []
            for basket in history_baskets:
                for item in basket_items_dict[basket]:
                    history_items.append(item)
            history_items = set(history_items)
            score = 0
            for item in basket_items_dict[next_basket]:
                if item in history_items:
                    score +=1
            rep_score[user] += score/len(basket_items_dict[next_basket])
        rep_score[user]/=len(baskets)
    return rep_score

def repeat_score_user(train_baskets):
    user_baskets = train_baskets[['user_id','date','basket_id']].drop_duplicates().\
        sort_values(['user_id','date'],ascending=True).groupby(['user_id'])['basket_id'].apply(list).reset_index()
    user_baskets_dict = dict(zip(user_baskets['user_id'],user_baskets['basket_id']))

    basket_items = train_baskets.groupby(['basket_id'])['item_id'].apply(list).reset_index()
    basket_items_dict = dict(zip(basket_items['basket_id'],basket_items['item_id']))

    rep_score = {}
    for user in user_baskets_dict:
        baskets = user_baskets_dict[user]
        rep_score[user] = 0
        for i in range(len(baskets)):
            next_basket = baskets[i]
            history_baskets = baskets[:i]

            history_items = []
            for basket in history_baskets:
                for item in basket_items_dict[basket]:
                    history_items.append(item)
            history_items = set(history_items)
            score = 0
            for item in basket_items_dict[next_basket]:
                if item in history_items:
                    score +=1
            rep_score[user] += score/len(basket_items_dict[next_basket])
        rep_score[user]/=len(baskets)
    return rep_score

In [ ]:
info = train(model_new, epochs=2, calc_recall=False, checkpoint=True)

In [ ]:
print(2)

2


In [ ]:
#info = train(model_new, epochs=2, calc_recall=True, checkpoint=True)

In [ ]:
#'/kaggle/input/tr-weights/epoch_recanet_best_model_dunnhumby.pth'

In [ ]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# model_new = ReCaNet(num_items=dataset.num_items, item_embed_size=item_embed_size, num_users=dataset.num_users, 
#                 user_embed_size=user_embed_size, history_len = history_len, h1 = h1,h2 = h2, h3 = h3,
#                 h4 = h4, h5 = h5).to(device)

# path_best = '/kaggle/input/tr-weights/epoch_recanet_best_model_dunnhumby.pth'
# #'/kaggle/input/lstm-weights/epoch_recanet_best_model_dunnhumby-lstm.pth'
# model_new.load_state_dict(torch.load(path_best))

In [ ]:
#best_model = model_new

In [ ]:
# import torch.nn.functional as F
# test_baskets = pd.read_csv(path_test) 
# #path_test
# #pd.read_csv(path_test)
# user_test_baskets_df = test_baskets.groupby('user_id')['item_id'].apply(list).reset_index()
# user_test_baskets_dict = dict(zip( user_test_baskets_df['user_id'],user_test_baskets_df['item_id']))

# user_predictions = predict(best_model, test_loader)
# final_users = set(dataset.test_users).intersection(set(list(user_test_baskets_dict.keys())))
# print('predictions ready',len(user_predictions))
# print('number of final test users:',len(final_users))
# for k in [5,10,20,'B']:
#     print(k)
#     recall_scores = {}
#     ndcg_scores = {}
#     #zero = 0
#     for user in final_users:

#         top_items = []
#         if user in user_predictions:
#             top_items = user_predictions[user]
#         else:
#             zero+=1

#         if k == 'B':
#             recall_scores[user] = recall_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
#             ndcg_scores[user] = ndcg_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
#         else:
#             recall_scores[user] = recall_k(user_test_baskets_dict[user],top_items,k)
#             ndcg_scores[user] = ndcg_k(user_test_baskets_dict[user],top_items,k)
#     #print(zero)
#     print('recall:',np.mean(list(recall_scores.values())))
#     print('ndcg:',np.mean(list(ndcg_scores.values())))

In [ ]:
# test_baskets = pd.read_csv(path_test) 
# #path_test
# #pd.read_csv(path_test)
# user_test_baskets_df = test_baskets.groupby('user_id')['item_id'].apply(list).reset_index()
# user_test_baskets_dict = dict(zip( user_test_baskets_df['user_id'],user_test_baskets_df['item_id']))

# user_predictions = predict(best_model, test_loader)
# final_users = set(dataset.test_users).intersection(set(list(user_test_baskets_dict.keys())))
# print('predictions ready',len(user_predictions))
# print('number of final test users:',len(final_users))
# for k in [5,10,20,'B']:
#     print(k)
#     recall_scores = {}
#     ndcg_scores = {}
#     #zero = 0
#     for user in final_users:

#         top_items = []
#         if user in user_predictions:
#             top_items = user_predictions[user]
#         else:
#             zero+=1

#         if k == 'B':
#             recall_scores[user] = recall_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
#             ndcg_scores[user] = ndcg_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
#         else:
#             recall_scores[user] = recall_k(user_test_baskets_dict[user],top_items,k)
#             ndcg_scores[user] = ndcg_k(user_test_baskets_dict[user],top_items,k)
#     #print(zero)
#     print('recall:',np.mean(list(recall_scores.values())))
#     print('ndcg:',np.mean(list(ndcg_scores.values())))

In [ ]:
# test_baskets = pd.read_csv(path_test) 
# #path_test
# #pd.read_csv(path_test)
# user_test_baskets_df = test_baskets.groupby('user_id')['item_id'].apply(list).reset_index()
# user_test_baskets_dict = dict(zip( user_test_baskets_df['user_id'],user_test_baskets_df['item_id']))

# user_predictions = predict(best_model, test_loader)
# final_users = set(dataset.test_users).intersection(set(list(user_test_baskets_dict.keys())))
# print('predictions ready',len(user_predictions))
# print('number of final test users:',len(final_users))
# for k in [5,10,20,'B']:
#     print(k)
#     recall_scores = {}
#     ndcg_scores = {}
#     #zero = 0
#     for user in final_users:

#         top_items = []
#         if user in user_predictions:
#             top_items = user_predictions[user]
#         else:
#             zero+=1

#         if k == 'B':
#             recall_scores[user] = recall_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
#             ndcg_scores[user] = ndcg_k(user_test_baskets_dict[user],top_items,len(user_test_baskets_dict[user]))
#         else:
#             recall_scores[user] = recall_k(user_test_baskets_dict[user],top_items,k)
#             ndcg_scores[user] = ndcg_k(user_test_baskets_dict[user],top_items,k)
#     #print(zero)
#     print('recall:',np.mean(list(recall_scores.values())))
#     print('ndcg:',np.mean(list(ndcg_scores.values())))